Tema escolhido: Postos de distribuição, Caminhões para carga nas docas

In [46]:
import simpy
import numpy as np
from scipy.stats import norm
from scipy.stats import expon

Listas de horários de chegada, saída dos postos, horários de chegada, saída das filas, tempo na fila e tamanho das filas

In [47]:
# listas de horários de chegada e saída dos postos
chegadas, saidas = [],[]

# listas de horários de chegada e saída das filas
in_queue, in_system  = [],[]

total, naDoca = [], []

# tempo na fila e tamanho das filas
horarios_nas_filas, tamanho_da_fila = [],[]


Preparando o ambiente e variáveis necessárias

In [48]:
# preparamos o ambiente 
env = simpy.Environment()
TEMPO_DE_SIMULACAO  = 1000 
np.random.seed(seed = 1)   
QUANTIDADE_DE_POSTOS = 10     
MEDIA_CHEGADA_CAMINHOES = 3
MEDIA_DE_CARGA_CAMINHOES = 9.0
DESVIO_DE_CARGA_CAMINHOES = 3.0
posto_de_carga = simpy.Resource(env, capacity = QUANTIDADE_DE_POSTOS)

Função para salvar informações da fila

In [49]:
def salva_info_da_fila(env, posto):
    horario_medicao = env.now
    tamanho_da_fila_agora = len(posto.queue)
    horarios_nas_filas.append(horario_medicao)
    tamanho_da_fila.append(tamanho_da_fila_agora)
    return horario_medicao

Distribuição exponencial para a chegada dos caminhões

In [50]:
def distribuicao_chegada_de_caminhoes():
  tempo_do_proximo_caminhao = expon.rvs(scale = MEDIA_CHEGADA_CAMINHOES, size = 1)
  return tempo_do_proximo_caminhao

Função para calcular tempo no sistema

In [51]:
def calcula_tempo_no_sistema(env, horario_chegada):
  horario_saida = env.now
  saidas.append(horario_saida)
  tempo_total = horario_saida - horario_chegada
  in_system.append(tempo_total)

Funções para chegada dos caminhoes na fila, para tempo de carga deles e para carregar

In [52]:
# Função que define a chegada de caminhões
# usando alguma funcao de distribuicao de probabilidade,
# define a ordem e tempo de chegada dos caminhoes
def chegada_dos_caminhoes(env):
    # ID para cada caminhão
    caminhao_id = 0
    while True:
       ## tempo de chegada do proximo caminhao
       tempo_do_proximo_caminhao = distribuicao_chegada_de_caminhoes()
       # espera pelo próximo caminhao
       yield env.timeout(tempo_do_proximo_caminhao)

       # caminhao chegou, marca o tempo e guarda o tempo de chegada
       tempo_de_chegada = env.now
       chegadas.append(tempo_de_chegada)
       caminhao_id += 1
       print('%3d chegou no posto em %.2f' % (caminhao_id, tempo_de_chegada))
       
       # executa a carga
       env.process(carga(env, caminhao_id, tempo_de_chegada))

# pega o tempo de carga do caminhao
def tempo_de_carga_caminhao():
  return norm.rvs(loc = MEDIA_DE_CARGA_CAMINHOES, 
                  scale = DESVIO_DE_CARGA_CAMINHOES, 
                  size = 1)

# executa a carga do caminhao
def carga(env, caminhao_id, horario_chegada):
    with posto_de_carga.request() as req:
        print('%3d entrou na fila para carregar em %.2f' % (caminhao_id, env.now))
        horario_entrada_da_fila = salva_info_da_fila(env, posto_de_carga)
        yield req # espera o posto ser liberado

        print('%3d saiu da fila e foi para a doca %.2f' % (caminhao_id, env.now))
        horario_saida_da_fila = salva_info_da_fila(env, posto_de_carga)

        # tempo que ficou na fila
        tempo_na_fila = horario_saida_da_fila - horario_entrada_da_fila
        in_queue.append(tempo_na_fila)

        # Execução da carga do caminhão
        tempo_carga = tempo_de_carga_caminhao()

        queue_carga = tempo_na_fila + tempo_carga



        yield env.timeout(tempo_carga)
        print('%3d permaneceu na doca por %.2f' % (caminhao_id, tempo_carga))
        
        total.append(queue_carga)
        naDoca.append(tempo_carga)
        # tempo total da operacao de carga + fila
        calcula_tempo_no_sistema(env, horario_chegada)
        

Rodamos a simulação

In [53]:
env.process(chegada_dos_caminhoes(env))
# Roda a simulação
env.run(until = TEMPO_DE_SIMULACAO)

  1 chegou no posto em 1.62
  1 entrou na fila para carregar em 1.62
  1 saiu da fila e foi para a doca 1.62
  2 chegou no posto em 5.44
  2 entrou na fila para carregar em 5.44
  2 saiu da fila e foi para a doca 5.44
  3 chegou no posto em 6.96
  3 entrou na fila para carregar em 6.96
  3 saiu da fila e foi para a doca 6.96
  1 permaneceu na doca por 7.42
  4 chegou no posto em 9.28
  4 entrou na fila para carregar em 9.28
  4 saiu da fila e foi para a doca 9.28
  5 chegou no posto em 9.97
  5 entrou na fila para carregar em 9.97
  5 saiu da fila e foi para a doca 9.97
  2 permaneceu na doca por 5.78
  4 permaneceu na doca por 6.72
  6 chegou no posto em 16.28
  6 entrou na fila para carregar em 16.28
  6 saiu da fila e foi para a doca 16.28
  7 chegou no posto em 16.73
  7 entrou na fila para carregar em 16.73
  7 saiu da fila e foi para a doca 16.73
  8 chegou no posto em 17.40
  8 entrou na fila para carregar em 17.40
  8 saiu da fila e foi para a doca 17.40
  6 permaneceu na doca 

In [68]:
tempo_medio = np.sum(naDoca)
tempo_medio = np.divide(tempo_medio, len(naDoca))

print(f"Tempo Medio: {tempo_medio: .2f}")

Tempo Medio:  9.18
